In [1]:
from brickmapper import Mapper
from rdflib import BRICK
import urllib.request
import csv
import numpy as np
from pprint import pprint
from brickmapper.bedes_parser.bedes_parser import BedesParser


In [2]:
bedes = BedesParser('v2.5')

defns = [
    {
        "name": term["Term"],
        "category": term["Category"],
        "term_definition": term["Term-Definition"]
    }
    for term in bedes.terms
]

defns

[{'name': 'Assessment Compliance Target Date',
  'category': 'Premises',
  'term_definition': 'Date a premises is expected to achieve assessment recognition, including in the appropriate cases, third party verification'},
 {'name': 'Assessment Eligibility',
  'category': 'Premises',
  'term_definition': 'Eligibility of a premises for assessment recognition.'},
 {'name': 'Assessment Level',
  'category': 'Premises',
  'term_definition': 'Value from assessment programs that produce a descriptive (rather than numeric) rating, such as LEED or NGBS.'},
 {'name': 'Assessment Program',
  'category': 'Premises',
  'term_definition': 'Program which issues energy labels, ratings, or sustainability certifications.'},
 {'name': 'Assessment Program Organization',
  'category': 'Premises',
  'term_definition': 'The name of the body or group providing the verification or certification assessment program. More than one can apply to a premises.'},
 {'name': 'Assessment Program URL',
  'category': 'Prem

In [5]:
# create a new Mapper object which knows about the definitions. This will
# access OpenAI to compute embeddings if necessary
m = Mapper(defns, "bedes_proto.index")

# define which set of Brick classes we want to match against
# all points which aren't parameters (these have similar enough names it can confuse the mapping)
points = m.get_brick_classes(BRICK.Point, [BRICK.Parameter])
# all equipment in Brick
equips = m.get_brick_classes(BRICK.Equipment)
# concatenate these two lists of Brick classes
bcs = points + equips

bcs

2024-12-24 12:52:36.099 | INFO     | brickmapper:populate_external_embeddings:72 - No local 'self.external_index_file' found. Computing embeddings for external concepts

 0%|                                                                                                                                                                         | 0/659 [00:00<?, ?it/s]

RateLimitError: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.

In [4]:
# compute a 'stable' mapping (1:1 mapping)
stable = m.get_mapping(bcs, allow_collisions=False)
# compute an 'unstable' mapping that allows 1:n and n:1 matches
unstable = m.get_mapping(bcs, allow_collisions=True)

# loop through and print the stable/unstable matches
for k,v in unstable.items():
    print(f"{k}:\n\tunstable: {v}\n\tstable: {stable.get(k)}")

NameError: name 'm' is not defined